# Neurona Artificial Simple desde Cero (Descenso del Gradiente "a pedal")

**Presentado por:**
*   [Nombre del Profesor/Presentador]
*   [Institución/Curso]
*   [Fecha]

In [ ]:
# Instalación de librerías necesarias (si no están presentes en el entorno de Colab)
!pip install pandas openpyxl

In [ ]:
import pandas as pd

# Cargar el archivo de datos
try:
    df = pd.read_excel('datos_sesion3.xlsx')
except FileNotFoundError:
    print("Error: El archivo 'datos_sesion3.xlsx' no fue encontrado. Asegúrate de que esté en el directorio correcto.")
    # Crear un DataFrame vacío para evitar errores posteriores si el archivo no se encuentra
    df = pd.DataFrame(columns=['frente', 'profundidad', 'precio'])


# Extraer las series de datos
# Asegurarse de que las columnas existen antes de intentar acceder a ellas
frentes = df['frente'] if 'frente' in df.columns else pd.Series(dtype='float64')
profundidades = df['profundidad'] if 'profundidad' in df.columns else pd.Series(dtype='float64')
precios = df['precio'] if 'precio' in df.columns else pd.Series(dtype='float64')

# Mostrar las primeras filas del DataFrame y las series para verificar
print("DataFrame cargado:")
print(df.head())
print("\nFrentes:")
print(frentes.head())
print("\nProfundidades:")
print(profundidades.head())
print("\nPrecios:")
print(precios.head())

## Estructura de la Neurona y Descenso del Gradiente

Nuestra neurona artificial simple tomará dos entradas (frente y profundidad del terreno) y intentará predecir el precio. La estructura es:

*   **Entradas (features):**
    *   `x1`: frente del terreno
    *   `x2`: profundidad del terreno
*   **Pesos (weights):**
    *   `w1`: peso asociado a `x1`
    *   `w2`: peso asociado a `x2`
*   **Sesgo (bias):**
    *   `b`: término de sesgo independiente

**Cálculo de la Predicción (Salida de la Neurona):**

La predicción (`y_pred`) se calcula como una combinación lineal de las entradas y los pesos, más el sesgo:
`y_pred = (x1 * w1) + (x2 * w2) + b`

**Función de Costo (Error Cuadrático Medio - MSE):**

Para medir qué tan bien está funcionando nuestra neurona, usamos la función de costo MSE. El objetivo es minimizar este costo. La fórmula, usando `m` como el número total de ejemplos de entrenamiento, es:
`J(w1, w2, b) = (1 / (2 * m)) * Σ( (y_real_i - y_pred_i)^2 )`
donde el sumatorio (Σ) es sobre todos los ejemplos de entrenamiento.

**Descenso del Gradiente (Actualización de Pesos):**

Para minimizar el costo, ajustamos `w1`, `w2`, y `b` iterativamente usando el descenso del gradiente. Esto implica calcular las derivadas parciales de la función de costo con respecto a cada parámetro:

*   **Derivada con respecto a `w1`:**
    `dJ/dw1 = (1 / m) * Σ( (y_pred_i - y_real_i) * x1_i )`
    (Nota: A veces se usa `(y_real_i - y_pred_i) * (-x1_i) / m` que es equivalente)

*   **Derivada con respecto a `w2`:**
    `dJ/dw2 = (1 / m) * Σ( (y_pred_i - y_real_i) * x2_i )`
    (Nota: A veces se usa `(y_real_i - y_pred_i) * (-x2_i) / m` que es equivalente)

*   **Derivada con respecto a `b`:**
    `dJ/db = (1 / m) * Σ( (y_pred_i - y_real_i) * 1 )`
    (Nota: A veces se usa `(y_real_i - y_pred_i) * (-1) / m` que es equivalente)

**Reglas de Actualización:**

Los pesos y el sesgo se actualizan en cada iteración (época) usando una tasa de aprendizaje (`η` o `alpha`):
`w1 = w1 - η * (dJ/dw1)`
`w2 = w2 - η * (dJ/dw2)`
`b = b - η * (dJ/db)`

Este proceso se repite durante un número determinado de épocas, con el objetivo de que los pesos converjan a valores que minimicen el error.

In [ ]:
import numpy as np

def calculo_neurona(frente_i, profundidad_i, w1, w2, b):
    """Calcula la salida de la neurona (predicción)."""
    # y_pred = (x1 * w1) + (x2 * w2) + b
    return (frente_i * w1) + (profundidad_i * w2) + b

def funcion_costo_mse(frentes, profundidades, precios, w1, w2, b):
    """Calcula el Error Cuadrático Medio (MSE)."""
    m = len(frentes)
    if m == 0:
        return 0
    
    total_error = 0
    for i in range(m):
        y_pred_i = calculo_neurona(frentes.iloc[i], profundidades.iloc[i], w1, w2, b)
        y_real_i = precios.iloc[i]
        total_error += (y_real_i - y_pred_i)**2
        
    mse = total_error / (2 * m)
    return mse

def derivada_w1(frentes, profundidades, precios, w1, w2, b):
    """Calcula la derivada parcial del costo MSE con respecto a w1."""
    m = len(frentes)
    if m == 0:
        return 0
        
    total_derivada_w1 = 0
    for i in range(m):
        y_pred_i = calculo_neurona(frentes.iloc[i], profundidades.iloc[i], w1, w2, b)
        y_real_i = precios.iloc[i]
        # (real - pred) * (-X_respectiva)
        total_derivada_w1 += (y_real_i - y_pred_i) * (-frentes.iloc[i])
        
    return total_derivada_w1 / m

def derivada_w2(frentes, profundidades, precios, w1, w2, b):
    """Calcula la derivada parcial del costo MSE con respecto a w2."""
    m = len(frentes)
    if m == 0:
        return 0
        
    total_derivada_w2 = 0
    for i in range(m):
        y_pred_i = calculo_neurona(frentes.iloc[i], profundidades.iloc[i], w1, w2, b)
        y_real_i = precios.iloc[i]
        # (real - pred) * (-X_respectiva)
        total_derivada_w2 += (y_real_i - y_pred_i) * (-profundidades.iloc[i])
        
    return total_derivada_w2 / m

def derivada_b(frentes, profundidades, precios, w1, w2, b):
    """Calcula la derivada parcial del costo MSE con respecto a b."""
    m = len(frentes)
    if m == 0:
        return 0
        
    total_derivada_b = 0
    for i in range(m):
        y_pred_i = calculo_neurona(frentes.iloc[i], profundidades.iloc[i], w1, w2, b)
        y_real_i = precios.iloc[i]
        # (real - pred) * (-1)
        total_derivada_b += (y_real_i - y_pred_i) * (-1)
        
    return total_derivada_b / m

# Prueba rápida de las funciones (opcional, se puede comentar o eliminar después)
# print("Funciones definidas. Realizando una prueba rápida...")
# w1_test, w2_test, b_test = 0.1, 0.2, 0.05
# if not frentes.empty and not profundidades.empty and not precios.empty:
#     y_pred_test = calculo_neurona(frentes.iloc[0], profundidades.iloc[0], w1_test, w2_test, b_test)
#     print(f"Predicción para el primer dato con pesos de prueba: {y_pred_test}")
#     costo_test = funcion_costo_mse(frentes, profundidades, precios, w1_test, w2_test, b_test)
#     print(f"Costo MSE con pesos de prueba: {costo_test}")
#     dw1_test = derivada_w1(frentes, profundidades, precios, w1_test, w2_test, b_test)
#     print(f"Derivada w1 con pesos de prueba: {dw1_test}")
#     dw2_test = derivada_w2(frentes, profundidades, precios, w1_test, w2_test, b_test)
#     print(f"Derivada w2 con pesos de prueba: {dw2_test}")
#     db_test = derivada_b(frentes, profundidades, precios, w1_test, w2_test, b_test)
#     print(f"Derivada b con pesos de prueba: {db_test}")
# else:
#     print("No hay datos cargados para realizar la prueba rápida de funciones.")

In [ ]:
def proceso_entrenamiento(frentes_train, profundidades_train, precios_train, tasa_aprendizaje_n, num_epocas):
    """
    Implementa el descenso del gradiente para entrenar la neurona.
    Inicializa los pesos w1, w2, b en cero.
    Devuelve los pesos finales y una lista con el error MSE por época.
    """
    w1 = 0.0
    w2 = 0.0
    b = 0.0
    
    historial_error_mse = []
    
    if frentes_train.empty or profundidades_train.empty or precios_train.empty:
        print("Datos de entrenamiento vacíos. No se puede proceder con el entrenamiento.")
        return w1, w2, b, historial_error_mse

    for epoca in range(num_epocas):
        # Calcular derivadas
        dj_dw1 = derivada_w1(frentes_train, profundidades_train, precios_train, w1, w2, b)
        dj_dw2 = derivada_w2(frentes_train, profundidades_train, precios_train, w1, w2, b)
        dj_db = derivada_b(frentes_train, profundidades_train, precios_train, w1, w2, b)
        
        # Actualizar pesos y sesgo
        w1 = w1 - tasa_aprendizaje_n * dj_dw1
        w2 = w2 - tasa_aprendizaje_n * dj_dw2
        b = b - tasa_aprendizaje_n * dj_db
        
        # Calcular y registrar el costo MSE para esta época
        costo_actual = funcion_costo_mse(frentes_train, profundidades_train, precios_train, w1, w2, b)
        historial_error_mse.append(costo_actual)
        
    return w1, w2, b, historial_error_mse

# Prueba de la función de entrenamiento (opcional)
# print("Función de entrenamiento definida.")
# if not frentes.empty:
#     tasa_prueba = 0.0001 # Puede necesitar ajuste según los datos
#     epocas_prueba = 10
#     print(f"Iniciando prueba de entrenamiento con {epocas_prueba} épocas y tasa {tasa_prueba}...")
#     w1_final_prueba, w2_final_prueba, b_final_prueba, errores_prueba = proceso_entrenamiento(frentes, profundidades, precios, tasa_prueba, epocas_prueba)
#     print(f"Pesos finales de prueba: w1={w1_final_prueba}, w2={w2_final_prueba}, b={b_final_prueba}")
#     if errores_prueba:
#         print(f"Error MSE inicial (prueba): {errores_prueba[0]}")
#         print(f"Error MSE final (prueba): {errores_prueba[-1]}")
# else:
#     print("No hay datos cargados para probar la función de entrenamiento.")

In [ ]:
# Parámetros de entrenamiento
# Estos valores pueden necesitar ajuste dependiendo de la escala de los datos y la convergencia observada.
# Una tasa de aprendizaje muy grande puede hacer que el error diverja.
# Una tasa muy pequeña puede hacer que el entrenamiento sea muy lento.
# Normalizar los datos de entrada (frentes, profundidades, precios) puede ayudar a encontrar
# una buena tasa de aprendizaje más fácilmente y mejorar la convergencia.
# Por ahora, usaremos valores que podrían funcionar con datos no normalizados,
# pero esto es altamente dependiente de la magnitud de los precios y las características.

# Dada la magnitud típica de los precios (ej: 250000) y las características (ej: 10, 20),
# el gradiente puede ser muy grande. (y_real - y_pred) * (-X).
# Si (y_real - y_pred) es del orden de 10000 y X es 10, el gradiente es 100000.
# Si la tasa es 0.01, el cambio de peso es 1000, lo cual es enorme.
# Se necesita una tasa de aprendizaje MUY pequeña.

# Intentemos con valores iniciales, pueden necesitarse ajustes:
tasa_aprendizaje = 1e-7 # Ejemplo: 0.0000001. Si los precios son muy altos, podría ser incluso menor.
                        # O considerar normalizar los datos.
num_epocas_entrenamiento = 1000 # Empezar con 1000, se puede aumentar.

print(f"Iniciando entrenamiento con tasa de aprendizaje: {tasa_aprendizaje} y {num_epocas_entrenamiento} épocas.")
print("---")
print("Si los datos no han sido cargados (ej. 'frentes' está vacío), el entrenamiento no se ejecutará.")
print("Asegúrese de que 'datos_sesion3.xlsx' está presente y las celdas anteriores se han ejecutado.")
print("---")

# Asegurarse de que las series de datos no están vacías antes de entrenar
if not frentes.empty and not profundidades.empty and not precios.empty:
    w1_final, w2_final, b_final, historial_mse = proceso_entrenamiento(
        frentes, 
        profundidades, 
        precios, 
        tasa_aprendizaje, 
        num_epocas_entrenamiento
    )

    print(f"Entrenamiento completado.")
    print(f"Pesos finales:")
    print(f"  w1 (peso para 'frente'): {w1_final:.4f}")
    print(f"  w2 (peso para 'profundidad'): {w2_final:.4f}")
    print(f"  b (sesgo): {b_final:.4f}")
    print("\n--- Evolución del Error MSE ---")
    if historial_mse:
        print(f"Error MSE inicial (Época 1): {historial_mse[0]:.2f}")
        
        # Imprimir error cada 100 épocas
        for i in range(0, len(historial_mse), 100):
            if i == 0: continue # Ya se imprimió el inicial
            if i < len(historial_mse):
                 print(f"Error MSE en Época {i+1}: {historial_mse[i]:.2f}")
        
        print(f"Error MSE final (Época {num_epocas_entrenamiento}): {historial_mse[-1]:.2f}")
    else:
        print("No se generó historial de MSE (posiblemente datos vacíos).")
else:
    print("Entrenamiento omitido porque los datos (frentes, profundidades, precios) no están cargados.")
    # Asignar valores por defecto para que el resto del notebook no falle si se ejecuta sin datos
    w1_final, w2_final, b_final = 0,0,0
    historial_mse = []


In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

if historial_mse:
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epocas_entrenamiento + 1), historial_mse)
    plt.xlabel("Época")
    plt.ylabel("Error Cuadrático Medio (MSE)")
    plt.title("Evolución del Error MSE durante el Entrenamiento")
    plt.grid(True)
    plt.show()
else:
    print("No hay historial de MSE para graficar. Asegúrese de que el entrenamiento se haya ejecutado con datos.")


## Conclusiones de la Práctica

Esta práctica nos ha permitido construir una neurona artificial desde sus componentes más básicos, implementando el algoritmo de descenso del gradiente "a pedal". A través de este ejercicio, hemos podido observar directamente:

*   **El Flujo de Datos:** Cómo las entradas (frente, profundidad) se combinan con los pesos y el sesgo para generar una predicción.
*   **La Importancia de la Función de Costo:** El MSE nos dio una medida cuantitativa de qué tan erradas estaban nuestras predicciones, guiando el aprendizaje.
*   **El Mecanismo del Descenso del Gradiente:** Vimos cómo el cálculo de las derivadas nos indica la dirección para ajustar los pesos y el sesgo con el fin de minimizar el error. La tasa de aprendizaje juega un papel crucial aquí; una tasa inadecuada puede impedir la convergencia o ralentizarla excesivamente.
*   **El Proceso Iterativo del Aprendizaje:** La repetición del cálculo de predicciones, errores, derivadas y actualizaciones de pesos a lo largo de múltiples épocas es fundamental para que la neurona "aprenda".
*   **Visualización del Aprendizaje:** La gráfica de MSE vs. Épocas es una herramienta visual poderosa para entender si el modelo está aprendiendo (el error disminuye) o si hay problemas (el error se estanca, aumenta o fluctúa erráticamente).

**Aprendizajes Clave:**

*   **Sensibilidad a los Hiperparámetros:** La elección de la tasa de aprendizaje y el número de épocas es crítica. En este ejercicio, vimos que con datos no normalizados y valores de salida (precios) grandes, se requiere una tasa de aprendizaje muy pequeña para evitar que los ajustes de los pesos sean demasiado grandes y el error diverja.
*   **Necesidad de Normalización (Potencial):** Aunque no se implementó aquí para mantener la simplicidad, en casos reales, normalizar las variables de entrada y/o salida puede hacer que el entrenamiento sea más estable y rápido, permitiendo el uso de tasas de aprendizaje más "estándar".
*   **Fundamentos para Redes Neuronales Más Complejas:** Entender esta neurona simple es el primer paso para comprender redes neuronales más profundas y complejas. Los principios de propagación hacia adelante (cálculo de la predicción) y retropropagación del error (aunque aquí simplificada al cálculo directo de derivadas para el descenso del gradiente) son fundamentales.

Este ejercicio práctico refuerza la comprensión teórica y demuestra cómo, con herramientas básicas de Python y un entendimiento de los conceptos matemáticos subyacentes, podemos implementar los bloques constructivos del aprendizaje automático.